In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from parameters import *
import matplotlib.pyplot as plt

TD=4
model1=gp.Model('solution_with_reg')
#variable declaration
A_BW_t=model1.addVars(24,lb=0,ub=A_max,vtype=GRB.INTEGER,name='A_BW_t')
A_IW_t=model1.addVars(24,lb=0,ub=A_max,vtype=GRB.INTEGER,name='A_IW_t')
A_R_t=model1.addVars(24,lb=0,ub=A_max,vtype=GRB.INTEGER,name='A_R_t')
L_BW_t=model1.addVars(24,lb=0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='L_BW_t')
L_IW_t=model1.addVars(24,lb=0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='L_IW_t')
P_res_t=model1.addVars(24,lb=0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='P_res_t')
P_grid_t=model1.addVars(24,lb=-GRB.INFINITY,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='P_grid_t')
W_res=model1.addVar(lb=0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='W_res')
delta_W_res=model1.addVar(lb=-GRB.INFINITY,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='delta_W_res')
#variables about reserve market and regulation support
P_r_t=model1.addVars(24,lb=0,ub=50,vtype=GRB.CONTINUOUS,name='P_r_t')
P_reg_t=model1.addVars(24,lb=0,ub=50,vtype=GRB.CONTINUOUS,name='P_reg_t')
#P_d_t=model1.addVars(24,lb=0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='P_d_t')
#constraints
#power balance constraint:DC's power supply comes from utility grid and PV generation
model1.addConstrs(1/1000*(P_idle+0.75*P_peak)*(A_BW_t[i]+A_IW_t[i]+A_R_t[i])+1/1000*(L_BW_t[i]+L_IW_t[i])*(P_peak-P_idle)/L_rate+alpha[i]*P_r_t[i]+beta[i]*P_reg_t[i]
                 ==P_grid_t[i]+P_res_t[i] for i in range(T))
#QoS constraint of interactive loads
model1.addConstrs(A_IW_t[i]>=L_IW_t[i]/(L_rate-1/C_DT) for i in range(T))
#interactive loads must be immediately satisfied
model1.addConstrs(L_IW_t[i]>=predicted_IWs[i] for i in range(T))
#enough processing resource for batch loads
model1.addConstrs(L_rate*A_BW_t[i]>=L_BW_t[i] for i in range(T))
#batch loads constraints
for j in range(T):
    model1.addConstr(gp.quicksum(L_BW_t[i] for i in range(0,j))<=gp.quicksum(predicted_BWs[i] for i in range(0,j)))
#assume TD=4,further modification allowed
for j in range(T-TD):
    model1.addConstr(gp.quicksum(L_BW_t[i] for i in range(0,j+TD))>=gp.quicksum(predicted_BWs[i] for i in range(0,j)))
for j in range(T-TD,T):
    model1.addConstr(gp.quicksum(L_BW_t[i] for i in range(0,T))>=gp.quicksum(predicted_BWs[i] for i in range(0,j)))

model1.addConstrs(A_R_t[i]<=redundant_ratio*A_max for i in range(T))
model1.addConstrs(A_IW_t[i]+A_BW_t[i]+1/redundant_ratio*A_R_t[i]>=A_max for i in range(T))
model1.addConstrs(A_IW_t[i]+A_BW_t[i]+A_R_t[i]<=A_max for i in range(T))
#green certificate related constraints
model1.addConstrs(P_res_t[i]<=P_res_max[i] for i in range(T))
model1.addConstrs(P_grid_t[i]<=P_grid_max for i in range(T))
model1.addConstrs(P_grid_t[i]>=-P_grid_max for i in range(T))
model1.addConstr(gp.quicksum(P_res_t[i] for i in range(T))==W_res)
model1.addConstr(W_res<=W_res_max)
model1.addConstr(delta_W_res==W_res_fix-W_res)
#auxiliary variable z
z=model1.addVar(vtype=GRB.BINARY,name='z')
M=1e5
model1.addConstr(delta_W_res<=delta_W_res_fix+M*z)
model1.addConstr(delta_W_res>=delta_W_res_fix-M*(1-z))

model1.setObjective(gp.quicksum(L_BW_t[i]*C_BW+L_IW_t[i]*C_IW for i in range(T))
                   +gp.quicksum(reserve_price[i]*P_r_t[i]+electricity_price[i]*P_r_t[i]*alpha[i] for i in range(T))
                   +gp.quicksum(reg_cap_price[i]*s_reg*P_reg_t[i]+reg_mil_price[i]*s_reg*R_mil*P_reg_t[i] for i in range(T))
                   -gp.quicksum(P_grid_t[i]*electricity_price[i] for i in range(T))
                   -C_GC*delta_W_res-C_punish*(delta_W_res-delta_W_res_fix)*z,GRB.MAXIMIZE)
model1.optimize()

P_r_t_res=np.array([P_r_t[i].X for i in range(T)])
P_reg_t_res=np.array([P_reg_t[i].X for i in range(T)])


A_BW_t_res=np.array([A_BW_t[i].X for i in range(T)])
A_IW_t_res=np.array([A_IW_t[i].X for i in range(T)])
A_R_t_res=np.array([A_R_t[i].X for i in range(T)])
L_BW_t_res=np.array([L_BW_t[i].X for i in range(T)])
L_IW_t_res=np.array([L_IW_t[i].X for i in range(T)])
P_res_t_res=np.array([P_res_t[i].X for i in range(T)])
P_grid_t_res=np.array([P_grid_t[i].X for i in range(T)])

24
Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-24
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 293 rows, 219 columns and 1247 nonzeros
Model fingerprint: 0x9bdb4186
Model has 1 quadratic objective term
Variable types: 146 continuous, 73 integer (1 binary)
Coefficient statistics:
  Matrix range     [9e-05, 1e+05]
  Objective range  [1e-01, 8e+03]
  QObjective range [5e+02, 5e+02]
  Bounds range     [1e+00, 4e+04]
  RHS range        [1e+01, 1e+05]
Found heuristic solution: objective 743456.95158
Presolve removed 213 rows and 156 columns
Presolve time: 0.00s
Presolved: 80 rows, 63 columns, 644 nonzeros
Found heuristic solution: objective 1469005.0927
Variable types: 32 continuous, 31 integer (0 binary)
Found heuristic solution: objective 1640070.6237

R

In [5]:
temp1,temp2=[],[]
for j in range(0,T-TD):
    temp1.append(sum(L_BW_t_res[i] for i in range(0,j+TD)))
    temp2.append(sum(predicted_BWs[i] for i in range(0,j)))

In [7]:
print(np.array(temp1)-np.array(temp2))

[11000. 15400. 19100. 15400. 24200. 25300. 25600. 25300. 24200. 23100.
 22800. 23100. 17400. 11700.  6000.     0.  6800.     0.     0. 15400.]
